In [86]:
from torch import nn, optim
from torchvision import transforms, datasets
from torchvision.utils import save_image

import numpy as np
import imageio as img
import math
import time
import os

In [87]:
# models
from lib import vgan

# helper functions
# from lib.helper import *
from lib.helper import *

In [88]:
# unlike tensorflow where we have to define the graphs and their operations, in pytorch we deine the network as a class
# i.e we just define the operations and the forward prop, the backprop is sorta in built

# we get this done by defining our class that inherits fromthe torchnn.module and hence the other functions
# such as backprop can be used

In [89]:
print("Status of GPU: ", torch.cuda.is_available())

Status of GPU:  True


In [90]:
# Hyper Parameters
USE_CUDA = True 
MODEL_TYPE = "DCGAN"

DATA_FOLDER = "./mnist"
DATA_NAME = DATA_FOLDER.split('/')[-1].upper()

# hyper parameters
bSize = 100
learning_rate = 0.0002

dwidth = dheight = 64
features = dwidth*dheight

# DCGAN params
channels = 1
img_size = dwidth
ndf = ngf = img_size
latent_dim = 100

# Number of steps to apply to the discriminator
d_steps = 1  # In Goodfellow et. al 2014 this variable is assigned to 1

# Number of epochs
num_epochs = 200

In [91]:
# custom logger from (https://raw.githubusercontent.com/diegoalejogm/gans/master/utils.py) 
from lib.utils import Logger

# Create logger instance
logger = Logger(model_name=MODEL_TYPE, data_name=DATA_NAME, cuda=USE_CUDA)

# Discriminator

In [92]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is ``(nc) x 64 x 64``
            nn.Conv2d(channels, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. ``(ndf) x 32 x 32``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. ``(ndf*2) x 16 x 16``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

# Generator Network

In [94]:
# https://arxiv.org/pdf/1511.06434, DCGAN
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.kernal_size = 4
        self.stride = 2
        self.padding = 1
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( latent_dim, ngf * 8, 
                               kernel_size=self.kernal_size, 
                               stride=1, 
                               padding=0, 
                               bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 
                               kernel_size=self.kernal_size, 
                               stride=self.stride, 
                               padding=self.padding, 
                               bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 
                               kernel_size=self.kernal_size, 
                               stride=self.stride, 
                               padding=self.padding, 
                               bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 
                               kernel_size=self.kernal_size, 
                               stride=self.stride, 
                               padding=self.padding, 
                               bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, channels, 
                               kernel_size=self.kernal_size, 
                               stride=self.stride, 
                               padding=self.padding, 
                               bias=False),
            nn.Tanh()
            
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

# Train Functions

In [96]:
def to_cuda(thing):
    if torch.cuda.is_available() and USE_CUDA: 
        return thing.cuda()
    return thing

In [97]:
def train_discriminator(optimizer, real_data, fake_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = discriminator(real_data)
    # Calculate error and backpropagate
    error_real = loss(prediction_real, to_cuda(real_data_target(real_data.size(0))))
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, to_cuda(fake_data_target(real_data.size(0))))
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error
    return error_real + error_fake, prediction_real, prediction_fake


In [98]:
def train_generator(optimizer, fake_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    
    # Calculate error and backpropagate
    error = loss(prediction, to_cuda(real_data_target(prediction.size(0))))
    error.backward()
    
    # Update weights with gradients
    optimizer.step()
    
    # Return error
    return error

# Data Processing

In [99]:
def get_custom_data(data_folder):
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Resize((dwidth, dheight)),
#          transforms.Grayscale(num_output_channels=1),
         transforms.Normalize((.5), (.5))
        ])

    return datasets.ImageFolder(root=data_folder, transform=compose)

In [100]:
# normalize data (https://github.com/soumith/ganhacks)
def mnist_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Resize((dwidth, dheight)),
         transforms.Normalize((.5), (.5))
        ])
    out_dir = '{}/dataset'.format(DATA_FOLDER)
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)

In [101]:
# Load data
data = mnist_data()

# custom data
# folder = "/home/tempocv/FirstGAN/alpha/AmplifiedDataset"
# data = get_custom_data(folder)

In [102]:
# Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=bSize, shuffle=True)
# Num batches
num_batches = len(data_loader)

# Defining Network Essentials

In [103]:
# Vanilla GAN
# discriminator = to_cuda(vgan.Discriminator(features))
# generator = to_cuda(vgan.Generator(bSize, features))

# DCGAN
discriminator = to_cuda(Discriminator())
generator = to_cuda(Generator())

In [104]:
# Optimizers
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)


# If we replace vᵢ = D(xᵢ) and yᵢ=1 ∀ i (for all i) in the BCE-Loss definition, 
# we obtain the loss related to the real-images. Conversely if we set vᵢ = D(G(zᵢ)) and yᵢ=0 ∀ i, \
# we obtain the loss related to the fake-images. 
# In the mathematical model of a GAN I described earlier, the gradient of this had to be ascended, 
# but PyTorch and most other Machine Learning frameworks usually minimize functions instead. 
# Since maximizing a function is equivalent to minimizing it’s negative, and the BCE-Loss term has a minus sign, 
# we don’t need to worry about the sign.
# Loss function
loss = nn.BCELoss()

# Actual Training

In [105]:
# for testing 
num_test_samples = 16
test_noise = noise(num_test_samples, bSize)


# for epoch in range(num_epochs):
#     for n_batch, (real_batch,_) in enumerate(data_loader):
#         N = real_batch.size(0)
#         # 1. Train Discriminator
#         real_data = to_cuda(Variable(images_to_vectors(real_batch, features)))
        
#         # Generate fake data and detach 
#         # (so gradients are not calculated for generator)
#         # can use.clone() as well but it will be a copy
#         # .detach() uses the same memory
#         fake_data = generator(to_cuda(noise(N, bSize))).detach()
        
#         # Train D
#         d_error, d_pred_real, d_pred_fake = \
#               train_discriminator(d_optimizer, real_data, fake_data)

#         # 2. Train Generator
#         # Generate fake data
#         fake_data = generator(to_cuda(noise(N, bSize)))
#         # Train G
        
#         g_error = train_generator(g_optimizer, fake_data)
#         # Log batch error
#         logger.log(d_error, g_error, epoch, n_batch, num_batches)
        
        
#         # Display Progress every few batches
#         if (n_batch) % 100 == 0: 
#             test_images = to_cuda(vectors_to_images(generator(test_noise), 1, dwidth, dheight))

#             logger.log_images(
#                 test_images, num_test_samples, 
#                 epoch, n_batch, num_batches
#             );
#             # Display status Logs
#             logger.display_status(
#                 epoch, num_epochs, n_batch, num_batches,
#                 d_error, g_error, d_pred_real, d_pred_fake
#             )

In [106]:
print(discriminator)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [107]:
print(generator)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [ ]:
# for testing 
num_test_samples = 12
test_noise = to_cuda(Variable(torch.Tensor(np.random.normal(0, 1, (bSize, latent_dim, 1, 1)))))

for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(data_loader):

        # Adversarial ground truths
        
        valid = to_cuda(Variable(torch.Tensor(imgs.shape[0], 1, 1, 1).fill_(1.0), requires_grad=False))
        fake = to_cuda(Variable(torch.Tensor(imgs.shape[0], 1, 1, 1).fill_(0.0), requires_grad=False))

        # Configure input

        real_imgs = to_cuda(Variable(imgs.type(torch.Tensor)))
   
        # -----------------
        #  Train Generator
        # -----------------

        g_optimizer.zero_grad()

        # Sample noise as generator input
        z = to_cuda(Variable(torch.Tensor(np.random.normal(0, 1, (bSize, latent_dim, 1, 1)))))

        # Generate a batch of images
        gen_imgs = generator(z)
        
        # Loss measures generator's ability to fool the discriminator
        g_loss = loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        g_optimizer.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        d_optimizer.zero_grad()
        # Measure discriminator's ability to classify real from generated samples
        real_loss = loss(discriminator(real_imgs), valid)
        fake_loss = loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        d_optimizer.step()


        # Display Progress every few batches
        n_batch = epoch * len(data_loader) + i
        if (n_batch) % 100 == 0: 
            test_images = vectors_to_images(generator(test_noise), 1, dwidth, dheight)

#             logger.log_images(
#                 test_images, num_test_samples, 
#                 epoch, n_batch, num_batches
#             );
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, i, len(data_loader),
                d_loss, g_loss, real_loss, fake_loss
            )
            save_image(gen_imgs.data[:25], "dc-images/%d.png" % n_batch, nrow=5, normalize=True)

Epoch: [0/200], Batch Num: [0/600]
Discriminator Loss: 0.4343, Generator Loss: 0.5919
D(x): 0.0244, D(G(z)): 0.8441
Epoch: [0/200], Batch Num: [100/600]
Discriminator Loss: 0.0005, Generator Loss: 7.1516
D(x): 0.0002, D(G(z)): 0.0008
Epoch: [0/200], Batch Num: [200/600]
Discriminator Loss: 0.0005, Generator Loss: 7.6424
D(x): 0.0004, D(G(z)): 0.0005
Epoch: [0/200], Batch Num: [300/600]
Discriminator Loss: 0.0006, Generator Loss: 10.3593
D(x): 0.0011, D(G(z)): 0.0000
Epoch: [0/200], Batch Num: [400/600]
Discriminator Loss: 0.0021, Generator Loss: 6.2463
D(x): 0.0013, D(G(z)): 0.0030
Epoch: [0/200], Batch Num: [500/600]
Discriminator Loss: 0.0003, Generator Loss: 8.3360
D(x): 0.0003, D(G(z)): 0.0003
Epoch: [1/200], Batch Num: [0/600]
Discriminator Loss: 0.0005, Generator Loss: 7.0981
D(x): 0.0002, D(G(z)): 0.0009
Epoch: [1/200], Batch Num: [100/600]
Discriminator Loss: 0.0003, Generator Loss: 8.1283
D(x): 0.0003, D(G(z)): 0.0003
Epoch: [1/200], Batch Num: [200/600]
Discriminator Loss: 0.

Epoch: [11/200], Batch Num: [400/600]
Discriminator Loss: 0.1583, Generator Loss: 1.9070
D(x): 0.0021, D(G(z)): 0.3146
Epoch: [11/200], Batch Num: [500/600]
Discriminator Loss: 0.0597, Generator Loss: 3.5071
D(x): 0.0222, D(G(z)): 0.0973
Epoch: [12/200], Batch Num: [0/600]
Discriminator Loss: 0.4215, Generator Loss: 8.9676
D(x): 0.8427, D(G(z)): 0.0003
Epoch: [12/200], Batch Num: [100/600]
Discriminator Loss: 0.0314, Generator Loss: 4.9302
D(x): 0.0415, D(G(z)): 0.0212
Epoch: [12/200], Batch Num: [200/600]
Discriminator Loss: 0.1322, Generator Loss: 3.3525
D(x): 0.1992, D(G(z)): 0.0651
Epoch: [12/200], Batch Num: [300/600]
Discriminator Loss: 0.0670, Generator Loss: 5.2154
D(x): 0.1209, D(G(z)): 0.0130
Epoch: [12/200], Batch Num: [400/600]
Discriminator Loss: 0.0805, Generator Loss: 3.3690
D(x): 0.0871, D(G(z)): 0.0738
Epoch: [12/200], Batch Num: [500/600]
Discriminator Loss: 0.6722, Generator Loss: 6.5969
D(x): 1.3408, D(G(z)): 0.0036
Epoch: [13/200], Batch Num: [0/600]
Discriminator 